In [1]:
import pandas as pd
import os

# Пути к файлам и соответствующие названия колонок
file_paths = {
    # 'bert': '/bert/sim-words5-bert.csv',
    'glove': '/glove_python/sim-words_sent_5_glove.csv',
    'word2vec': '/word2vec/similar_words_sent_5.csv'
}

# Создаем пустой DataFrame с колонкой 'word'
data = pd.DataFrame(columns=['word'])

# Обрабатываем каждый файл
for model_name, path in file_paths.items():
    # Читаем CSV-файл
    full_path = ".." + path
    df = pd.read_csv(full_path)
    
    # Переименовываем колонку Most_Similar_Word в название модели
    df = df.rename(columns={'Most_Similar_Word': model_name})
    
    # Если это первый файл, используем его слова как основу
    if data.empty:
        data['word'] = df['Word']
    
    # Добавляем данные в основной DataFrame
    data[model_name] = df[model_name]

In [3]:
import ast
# Функция для проверки, является ли значение пустым списком или NaN
def is_valid_similar_words(value):
    if pd.isna(value):  # проверяем NaN
        return False
    if isinstance(value, str):  # если данные хранятся как строки (например, "[('слово', 0.5), ...]")
        try:
            lst = ast.literal_eval(value)  # преобразуем строку в список
            return len(lst) > 0  # True, если список не пустой
        except (ValueError, SyntaxError):
            return False
    elif isinstance(value, list):  # если данные уже в формате списка
        return len(value) > 0
    else:
        return False  # на случай других форматов

# Применяем фильтрацию: оставляем строки, где ВСЕ столбцы с похожими словами не пусты
filtered_data = data[
    data['glove'].apply(is_valid_similar_words) &
    data['word2vec'].apply(is_valid_similar_words)
]

In [5]:
n = 25  # кол-во случайных строк

random_sample = filtered_data.sample(n=n, random_state=42)  # random_state для воспроизводимости

# Сохраняем в новый CSV-файл
output_file = 'random_sample_similar_words.csv'
random_sample.to_csv(output_file, index=False, encoding='utf-8')

In [ ]:
import pandas as pd
docs = pd.read_csv("random_sample_similar_words.csv")
print(docs)

             word                                              glove  \
0        недорого  [('развлекаться', 0.7729333639144897), ('<unk>...   
1           голод                    [('холод', 0.7239494919776917)]   
2      орфография                [('пунктуация', 0.861853301525116)]   
3  внимательность        [('сосредоточенность', 0.7358331680297852)]   
4       обсценный  [('нецензурный', 0.7586269974708557), ('лексик...   
5         трещина                   [('ворона', 0.7095763087272644)]   
6          кадмий                  [('селенид', 0.8604143857955933)]   
7             омс                    [('полис', 0.7021245956420898)]   
8         автомат               [('калашников', 0.7369362711906433)]   
9           жажда                  [('утолять', 0.8408418893814087)]   

                                            word2vec  
0  [('инк', 0.856656551361084), ('прожект', 0.855...  
1  [('фашист', 0.8560139536857605), ('холод', 0.8...  
2               [('пунктуация', 0.71656578